<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import messagebox
from tkinter.simpledialog import askstring
import pandas as pd
from extract import execute

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
from replace_DFI_3 import findIndex3
import os


# Rest of your import statements...


def add_row(treeview):
    # Prompt the user for data input
    data = askstring("Add Row", "Enter data separated by commas (,):")
    if data:
        data = data.split(',')
        # Add the data as a new row to the Treeview
        treeview.insert("", "end", values=data)

def delete_row(treeview):
    # Get the selected item and delete it
    selected_item = treeview.selection()
    if selected_item:
        treeview.delete(selected_item)

def update_data(treeview, column_idx, new_value):
    # Get the selected item
    selected_item = treeview.selection()
    if selected_item:
        # Update the specified column with the new value
        treeview.item(selected_item, values=(new_value,))

def create_and_display_visualizations(dataframes):
    # Example: Create a bar chart of Total Debit Amount and Total Credit Amount
    for name, df in dataframes.items():
        if "Total Debit Amount" in df.columns and "Total Credit Amount" in df.columns:
            plt.figure(figsize=(8, 6))
            plt.bar(["Total Debit Amount", "Total Credit Amount"],
                    [df["Total Debit Amount"].iloc[0], df["Total Credit Amount"].iloc[0]])
            plt.title(f"Total Debit vs Total Credit - {name}")
            plt.xlabel("Amount Type")
            plt.ylabel("Amount")
            plt.xticks(rotation=45)
            plt.show()

def display_dataframe_in_treeview(treeview, df):
    treeview.delete(*treeview.get_children())  # Clear existing data

    # Insert columns
    treeview["columns"] = list(df.columns)
    style = ttk.Style()
    style.configure("Treeview.Heading", font=('Arial', 12, 'bold'))

    for col in df.columns:
        treeview.heading(col, text=col, anchor="center")

        # Calculate the maximum width needed for the column based on data length
        max_width = max(df[col].astype(str).apply(len).max(), len(col))

        # Set the column width
        treeview.column(col, anchor="center", width=max_width + 20)  # Add some extra width

    # Insert rows
    for _, row in df.iterrows():
        treeview.insert("", "end", values=list(row))

def sort_data(treeview, column_name):
    global selected_column
    # Sort the data by the selected column
    df = pd.DataFrame(treeview.get_children(), columns=treeview["columns"])
    df[column_name] = df[column_name].str.replace('$', '').str.replace(',', '').astype(float)
    df = df.sort_values(by=column_name, ascending=True)  # Change ascending to False for descending order
    display_dataframe_in_treeview(treeview, df)
    # Change text color for the selected column
    if selected_column:
        treeview.tag_configure(selected_column, foreground="black")
    selected_column = column_name
    treeview.tag_configure(selected_column, foreground="blue")

def filter_data(treeview, column_name):
    global selected_column
    # Implement filtering based on the selected column (modify this part)
    # ...
    # Change text color for the selected column
    if selected_column:
        treeview.tag_configure(selected_column, foreground="black")
    selected_column = column_name
    treeview.tag_configure(selected_column, foreground="blue")

def reset_dataframe(treeview, original_df, original_column_widths):
    display_dataframe_in_treeview(treeview, original_df)

    for col, width in original_column_widths.items():
        treeview.column(col, width=width)


def export_dataframes_to_excel_with_filters(dataframes, file_path):
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for name, df in dataframes.items():
            df.to_excel(writer, sheet_name=name, index=False)

            # Get the xlsxwriter workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets[name]

            # Add autofilter to the sheet to enable filtering in Excel
            num_rows, num_cols = df.shape
            worksheet.autofilter(0, 0, num_rows, num_cols - 1)  # Exclude the index column
            # Here, (0, 0, num_rows, num_cols - 1) sets the autofilter range

            # Iterate through columns to set column width based on data length
            for idx, col in enumerate(df):
                max_length = max(df[col].astype(str).apply(len).max(), len(col))
                adjusted_width = (max_length + 2)  # Add some extra width
                worksheet.set_column(idx, idx, adjusted_width)

selected_column = None

def main():
    file_path = filedialog.askopenfilename()
    dataframes, res, file_path = execute(file_path)  # You need to define the execute function

    root = tk.Tk()
    root.title("Editable Dataframes")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    canvas = tk.Canvas(root)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    scrollbar_y = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
    canvas.configure(yscrollcommand=scrollbar_y.set)

    frame = ttk.Frame(canvas)
    canvas.create_window((0, 0), window=frame, anchor="nw")

    def update_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    frame.bind("<Configure>", update_scroll_region)

    treeviews = {}  # Dictionary to store treeviews for each dataframe

    for name, df in dataframes.items():
        dataframe_frame = ttk.Frame(frame, padding=5, relief=tk.RAISED, borderwidth=2)
        dataframe_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        dataframe_label = ttk.Label(dataframe_frame, text=name, font=('Arial', 14, 'bold'))
        dataframe_label.pack(pady=5)

        treeview = ttk.Treeview(dataframe_frame, show="headings", style="Treeview.Heading")
        treeview.pack(fill="both", expand=True)

        # Store original column widths
        original_column_widths = {col: 150 for col in df.columns}

        display_dataframe_in_treeview(treeview, df)

        for col in df.columns:
            treeview.column(col, anchor="center", width=original_column_widths[col])  # Set original width

        reset_button = tk.Button(dataframe_frame, text="Undo",
                                 command=lambda tv=treeview, df=df, widths=original_column_widths: reset_dataframe(tv,
                                                                                                                   df,
                                                                                                                   widths))
        reset_button.pack()

        # Create vertical scrollbar for the treeview
        scrollbar_y_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.VERTICAL, command=treeview.yview)
        scrollbar_y_treeview.pack(side=tk.RIGHT, fill=tk.Y)
        treeview.configure(yscrollcommand=scrollbar_y_treeview.set)

        # Bind mouse wheel event for vertical scrolling
        def treeview_scroll_y(event, treeview=treeview):
            treeview.yview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<MouseWheel>", treeview_scroll_y)

        # Create horizontal scrollbar for the treeview
        scrollbar_x_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.HORIZONTAL, command=treeview.xview)
        scrollbar_x_treeview.pack(side=tk.BOTTOM, fill=tk.X)
        treeview.configure(xscrollcommand=scrollbar_x_treeview.set)

        # Bind shift + mouse wheel event for horizontal scrolling
        def treeview_scroll_x(event, treeview=treeview):
            treeview.xview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<Shift-MouseWheel>", treeview_scroll_x)

        treeviews[dataframe_frame] = treeview  # Store the treeview in the dictionary

    update_scroll_region(None)
    root.lift()

    root.geometry(f"{screen_width}x{screen_height}")
    root.wm_state("zoomed")

    def canvas_scroll(event):
        if event.delta < 0:
            canvas.yview_scroll(1, "units")
        else:
            canvas.yview_scroll(-1, "units")

    def canvas_scroll_horizontal(event):
        if event.delta < 0:
            canvas.xview_scroll(1, "units")
        else:
            canvas.xview_scroll(-1, "units")

    root.bind_all("<MouseWheel>", canvas_scroll)
    root.bind_all("<Shift-MouseWheel>", canvas_scroll_horizontal)

    # Create a horizontal scrollbar at the bottom
    scrollbar_x = ttk.Scrollbar(root, orient=tk.HORIZONTAL, command=canvas.xview)
    scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)
    canvas.configure(xscrollcommand=scrollbar_x.set)

    add_button = tk.Button(root, text="Add Row", command=lambda: add_row(treeviews[current_dataframe]))
    delete_button = tk.Button(root, text="Delete Row", command=lambda: delete_row(treeviews[current_dataframe]))
    edit_button = tk.Button(root, text="Edit Data",
                            command=lambda: update_data(treeviews[current_dataframe], 1, "New Value"))

    add_button.pack()
    delete_button.pack()
    edit_button.pack()

    sort_button = tk.Button(root, text="Sort Amount", command=lambda: sort_data(treeviews[current_dataframe], "Amount"))
    filter_button = tk.Button(root, text="Filter Data",
                              command=lambda: filter_data(treeviews[current_dataframe], "Amount"))

    sort_button.pack()
    filter_button.pack()

    export_button = tk.Button(root, text="Export to Excel",
                              command=lambda: export_dataframes_to_excel_with_filters(dataframes, "achexcel_with_filters.xlsx"))
    export_button.pack()

    visualization_button = tk.Button(root, text="Generate Visualizations",
                                     command=lambda: create_and_display_visualizations(dataframes))
    visualization_button.pack()

    root.mainloop()


if __name__ == '__main__':
    main()
